# packages

In [19]:
import pandas as pd


In [20]:
df = pd.read_csv('../outputs_hhhl_ml1/before_good26_rest/ORCL/backtest/df_stats.csv')

In [21]:
df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11
0,Start,2000-01-10 00:00:00,2006-06-19 00:00:00,2000-01-10 00:00:00,2009-07-17 00:00:00,2000-01-10 00:00:00,2012-08-08 00:00:00,2000-01-10 00:00:00,2015-09-15 00:00:00,2000-01-10 00:00:00,2018-08-29 00:00:00,2000-01-10 00:00:00,2021-07-15 00:00:00
1,End,2006-06-16 00:00:00,2009-07-16 00:00:00,2009-07-16 00:00:00,2012-08-07 00:00:00,2012-08-07 00:00:00,2015-09-14 00:00:00,2015-09-14 00:00:00,2018-08-28 00:00:00,2018-08-28 00:00:00,2021-07-12 00:00:00,2021-07-12 00:00:00,2024-10-10 00:00:00
2,Period,1618 days 00:00:00,775 days 00:00:00,2393 days 00:00:00,772 days 00:00:00,3165 days 00:00:00,779 days 00:00:00,3944 days 00:00:00,745 days 00:00:00,4689 days 00:00:00,721 days 00:00:00,5410 days 00:00:00,816 days 00:00:00
3,Start Value,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
4,End Value,290.77508190136484,242.1327157507496,3209.085046269844,306.1137161032776,3462.3753343410394,240.99195732631208,6015.343803012408,211.483926360233,8837.467898317822,230.3408712190465,88960.23579451621,546.3912384357037


In [22]:
df = df.set_index('Unnamed: 0')
df.index.name = None
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
Start,2000-01-10 00:00:00,2006-06-19 00:00:00,2000-01-10 00:00:00,2009-07-17 00:00:00,2000-01-10 00:00:00,2012-08-08 00:00:00,2000-01-10 00:00:00,2015-09-15 00:00:00,2000-01-10 00:00:00,2018-08-29 00:00:00,2000-01-10 00:00:00,2021-07-15 00:00:00
End,2006-06-16 00:00:00,2009-07-16 00:00:00,2009-07-16 00:00:00,2012-08-07 00:00:00,2012-08-07 00:00:00,2015-09-14 00:00:00,2015-09-14 00:00:00,2018-08-28 00:00:00,2018-08-28 00:00:00,2021-07-12 00:00:00,2021-07-12 00:00:00,2024-10-10 00:00:00
Period,1618 days 00:00:00,775 days 00:00:00,2393 days 00:00:00,772 days 00:00:00,3165 days 00:00:00,779 days 00:00:00,3944 days 00:00:00,745 days 00:00:00,4689 days 00:00:00,721 days 00:00:00,5410 days 00:00:00,816 days 00:00:00
Start Value,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
End Value,290.77508190136484,242.1327157507496,3209.085046269844,306.1137161032776,3462.3753343410394,240.99195732631208,6015.343803012408,211.483926360233,8837.467898317822,230.3408712190465,88960.23579451621,546.3912384357037


In [23]:
transformed_rows = []

# Iterate over pairs of columns (train-test combinations)
for i in range(0, df.shape[1], 2):
    train_col = i
    test_col = i + 1
    
    # Create a dictionary to store values for the current combination
    row = {}
    
    # Set the combination number
    row['train_test_combination'] = i // 2
    
    # Add train and test statistics to the row with new column names
    for stat in df.index:
        row[f'train_{stat.lower().replace(" ", "_")}'] = df.loc[stat, str(train_col)]
        row[f'test_{stat.lower().replace(" ", "_")}'] = df.loc[stat, str(test_col)]
    
    # Append the row to the transformed rows list
    transformed_rows.append(row)

# Create a new DataFrame from the transformed rows
df_transformed = pd.DataFrame(transformed_rows)

# Show the transformed DataFrame
df_transformed

,train_test_combination,train_start,test_start,train_end,test_end,train_period,test_period,train_start_value,test_start_value,train_end_value,...,train_expectancy,test_expectancy,train_sharpe_ratio,test_sharpe_ratio,train_calmar_ratio,test_calmar_ratio,train_omega_ratio,test_omega_ratio,train_sortino_ratio,test_sortino_ratio
0,0,2000-01-10 00:00:00,2006-06-19 00:00:00,2006-06-16 00:00:00,2009-07-16 00:00:00,1618 days 00:00:00,775 days 00:00:00,100.0,100.0,290.77508190136484,...,1.2953679828558768,3.384112279779753,0.9756839526837449,0.9801424760667765,0.4264400149319753,1.7474382779664943,1.2500141435965373,1.329824079772141,1.565967117589784,1.576195107776711
1,1,2000-01-10 00:00:00,2009-07-17 00:00:00,2009-07-16 00:00:00,2012-08-07 00:00:00,2393 days 00:00:00,772 days 00:00:00,100.0,100.0,3209.085046269844,...,22.93211548103335,4.528573860999403,1.1068446392420674,1.0875104598300134,1.4647688624609343,2.977416892632335,1.300568895540914,1.3006680852840522,1.7738521632272795,1.7057075655968585
2,2,2000-01-10 00:00:00,2012-08-08 00:00:00,2012-08-07 00:00:00,2015-09-14 00:00:00,3165 days 00:00:00,779 days 00:00:00,100.0,100.0,3462.3753343410394,...,17.67555897420664,2.660225609930417,1.027140585967952,0.9343712296971625,0.643945804366052,2.6463472867014,1.2786615153935936,1.2692344002711233,1.6464526085642948,1.471991983487113
3,3,2000-01-10 00:00:00,2015-09-15 00:00:00,2015-09-14 00:00:00,2018-08-28 00:00:00,3944 days 00:00:00,745 days 00:00:00,100.0,100.0,6015.343803012408,...,20.1839801955619,1.5483878661143473,0.9920683664200489,0.8652358644939073,0.7117608292899108,2.601962417540662,1.268213106172227,1.2036592384997238,1.5697405152562804,1.2928957199716384
4,4,2000-01-10 00:00:00,2018-08-29 00:00:00,2018-08-28 00:00:00,2021-07-12 00:00:00,4689 days 00:00:00,721 days 00:00:00,100.0,100.0,8837.467898317822,...,28.689556640731386,1.7805704025102207,0.9392791425347292,0.9589317253604934,0.6073448208217426,3.0480355394576772,1.259466835264326,1.2219530348234586,1.4857411576343724,1.4775112224324105
5,5,2000-01-10 00:00:00,2021-07-15 00:00:00,2021-07-12 00:00:00,2024-10-10 00:00:00,5410 days 00:00:00,816 days 00:00:00,100.0,100.0,88960.23579451621,...,223.86917639872865,7.2629495165784945,0.987661908471438,2.3655884381551444,1.0821355600727034,7.418604272035989,1.2815938277479224,1.6463182392156857,1.5681374799895036,4.174920925332052
